In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, # GQN
                                                                   labels=next_states_labels))
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                     labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a')
    targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
#     eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
#     aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
    eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
    aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
#     #################################################### s'', Q' = ~env(s', ~a'): Repeatable 1
#     next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
#                               reuse=True)
#     next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states_logits, hidden_size=hidden_size, 
#                                    action_size=action_size, state_size=state_size, reuse=True)
#     aQs_logits = aQs2_logits # pervious one
#     aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
# #     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                     labels=tf.zeros_like(rates))) # min
# #     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                      labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a')
#     targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
#     eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
#     aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
#     #################################################### s'', Q' = ~env(s', ~a'): Repeatable 2
#     next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
#                               reuse=True)
#     next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states_logits, hidden_size=hidden_size, 
#                                    action_size=action_size, state_size=state_size, reuse=True)
#     aQs_logits = aQs2_logits # pervious one
#     aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
# #     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                     labels=tf.zeros_like(rates))) # min
# #     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                      labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a')
#     targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
#     eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
#     aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:22.0000 R:22.0000 rate:0.0440 aloss:0.7093 eloss:2.9360 aloss2:1.3826 exploreP:0.9978
Episode:1 meanR:21.0000 R:20.0000 rate:0.0400 aloss:0.7054 eloss:2.9687 aloss2:1.3751 exploreP:0.9959
Episode:2 meanR:17.6667 R:11.0000 rate:0.0220 aloss:0.7066 eloss:2.9956 aloss2:1.3505 exploreP:0.9948
Episode:3 meanR:19.0000 R:23.0000 rate:0.0460 aloss:0.7034 eloss:3.0391 aloss2:1.3636 exploreP:0.9925
Episode:4 meanR:20.0000 R:24.0000 rate:0.0480 aloss:0.7006 eloss:3.0057 aloss2:1.3513 exploreP:0.9901
Episode:5 meanR:19.0000 R:14.0000 rate:0.0280 aloss:0.6997 eloss:2.9768 aloss2:1.3602 exploreP:0.9888
Episode:6 meanR:18.7143 R:17.0000 rate:0.0340 aloss:0.7010 eloss:3.0081 aloss2:1.3552 exploreP:0.9871
Episode:7 meanR:17.8750 R:12.0000 rate:0.0240 aloss:0.7024 eloss:3.0374 aloss2:1.3405 exploreP:0.9859
Episode:8 meanR:17.5556 R:15.0000 rate:0.0300 aloss:0.7003 eloss:3.1231 aloss2:1.3373 exploreP:0.9845
Episode:9 meanR:17.5000 R:17.0000 rate:0.0340 aloss:0.6978 eloss:3.0134 aloss2:1.3

Episode:80 meanR:26.9877 R:22.0000 rate:0.0440 aloss:0.6804 eloss:1.7255 aloss2:1.8739 exploreP:0.8056
Episode:81 meanR:26.9390 R:23.0000 rate:0.0460 aloss:0.6792 eloss:1.7176 aloss2:1.8830 exploreP:0.8038
Episode:82 meanR:27.0602 R:37.0000 rate:0.0740 aloss:0.6707 eloss:1.7642 aloss2:1.8704 exploreP:0.8008
Episode:83 meanR:27.3452 R:51.0000 rate:0.1020 aloss:0.7146 eloss:1.7822 aloss2:1.8868 exploreP:0.7968
Episode:84 meanR:27.3412 R:27.0000 rate:0.0540 aloss:0.6828 eloss:1.6963 aloss2:1.9288 exploreP:0.7947
Episode:85 meanR:27.6047 R:50.0000 rate:0.1000 aloss:0.6869 eloss:1.6827 aloss2:1.9430 exploreP:0.7908
Episode:86 meanR:27.4598 R:15.0000 rate:0.0300 aloss:0.6840 eloss:1.6042 aloss2:1.9855 exploreP:0.7896
Episode:87 meanR:28.4318 R:113.0000 rate:0.2260 aloss:0.6798 eloss:1.6081 aloss2:1.9990 exploreP:0.7809
Episode:88 meanR:28.5618 R:40.0000 rate:0.0800 aloss:0.6818 eloss:1.5830 aloss2:2.0310 exploreP:0.7778
Episode:89 meanR:28.5333 R:26.0000 rate:0.0520 aloss:0.6976 eloss:1.5478

Episode:159 meanR:41.6400 R:80.0000 rate:0.1600 aloss:0.6987 eloss:-0.2935 aloss2:4.9419 exploreP:0.5675
Episode:160 meanR:42.4800 R:114.0000 rate:0.2280 aloss:0.7042 eloss:-0.3542 aloss2:5.0898 exploreP:0.5612
Episode:161 meanR:42.9700 R:67.0000 rate:0.1340 aloss:0.7132 eloss:-0.3699 aloss2:5.1816 exploreP:0.5575
Episode:162 meanR:43.8600 R:121.0000 rate:0.2420 aloss:0.7149 eloss:-0.4710 aloss2:5.3529 exploreP:0.5509
Episode:163 meanR:45.4900 R:180.0000 rate:0.3600 aloss:0.7032 eloss:-0.5479 aloss2:5.5837 exploreP:0.5413
Episode:164 meanR:45.4000 R:15.0000 rate:0.0300 aloss:0.7158 eloss:-0.6802 aloss2:5.7661 exploreP:0.5405
Episode:165 meanR:45.5400 R:32.0000 rate:0.0640 aloss:0.7239 eloss:-0.6418 aloss2:5.8062 exploreP:0.5388
Episode:166 meanR:45.5800 R:73.0000 rate:0.1460 aloss:0.7081 eloss:-0.6581 aloss2:5.9198 exploreP:0.5350
Episode:167 meanR:47.4900 R:205.0000 rate:0.4100 aloss:0.7221 eloss:-0.7621 aloss2:6.1522 exploreP:0.5243
Episode:168 meanR:47.9000 R:79.0000 rate:0.1580 alo

Episode:237 meanR:96.7200 R:120.0000 rate:0.2400 aloss:0.7387 eloss:-6.2708 aloss2:32.5060 exploreP:0.2449
Episode:238 meanR:97.2500 R:94.0000 rate:0.1880 aloss:0.7254 eloss:-7.0284 aloss2:32.8970 exploreP:0.2427
Episode:239 meanR:98.4300 R:151.0000 rate:0.3020 aloss:0.7161 eloss:-6.8431 aloss2:33.3570 exploreP:0.2392
Episode:240 meanR:99.1700 R:86.0000 rate:0.1720 aloss:0.7220 eloss:-7.2367 aloss2:33.7960 exploreP:0.2372
Episode:241 meanR:99.7600 R:131.0000 rate:0.2620 aloss:0.7485 eloss:-7.0659 aloss2:34.3296 exploreP:0.2343
Episode:242 meanR:100.6100 R:115.0000 rate:0.2300 aloss:0.7755 eloss:-5.4514 aloss2:34.6411 exploreP:0.2317
Episode:243 meanR:101.1300 R:100.0000 rate:0.2000 aloss:0.7328 eloss:-6.9762 aloss2:34.8932 exploreP:0.2295
Episode:244 meanR:101.9900 R:100.0000 rate:0.2000 aloss:0.7302 eloss:-6.5835 aloss2:35.0327 exploreP:0.2273
Episode:245 meanR:101.9400 R:78.0000 rate:0.1560 aloss:0.7310 eloss:-7.5412 aloss2:35.1973 exploreP:0.2256
Episode:246 meanR:101.8800 R:84.0000

Episode:314 meanR:107.2100 R:81.0000 rate:0.1620 aloss:0.7898 eloss:-3.5986 aloss2:45.4241 exploreP:0.1150
Episode:315 meanR:107.0500 R:101.0000 rate:0.2020 aloss:0.7035 eloss:-7.6460 aloss2:45.4858 exploreP:0.1140
Episode:316 meanR:107.2800 R:114.0000 rate:0.2280 aloss:0.7266 eloss:-7.3284 aloss2:45.2710 exploreP:0.1128
Episode:317 meanR:107.1400 R:69.0000 rate:0.1380 aloss:0.7148 eloss:-8.4605 aloss2:45.6862 exploreP:0.1121
Episode:318 meanR:107.1700 R:94.0000 rate:0.1880 aloss:0.7151 eloss:-6.0038 aloss2:45.5126 exploreP:0.1111
Episode:319 meanR:107.0500 R:85.0000 rate:0.1700 aloss:0.7305 eloss:-8.2124 aloss2:45.6235 exploreP:0.1103
Episode:320 meanR:106.9400 R:82.0000 rate:0.1640 aloss:0.7530 eloss:-5.8507 aloss2:45.4930 exploreP:0.1095
Episode:321 meanR:105.4600 R:91.0000 rate:0.1820 aloss:0.7148 eloss:-7.3195 aloss2:45.6053 exploreP:0.1086
Episode:322 meanR:105.4300 R:100.0000 rate:0.2000 aloss:0.7128 eloss:-8.7707 aloss2:45.9037 exploreP:0.1076
Episode:323 meanR:105.2300 R:92.00

...

Episode:545 meanR:105.6700 R:187.0000 rate:0.3740 aloss:0.7240 eloss:-7.3501 aloss2:56.4648 exploreP:0.0186
Episode:546 meanR:105.8900 R:115.0000 rate:0.2300 aloss:0.7246 eloss:-9.5868 aloss2:56.8401 exploreP:0.0185
Episode:547 meanR:106.3900 R:134.0000 rate:0.2680 aloss:0.6993 eloss:-9.1802 aloss2:56.8527 exploreP:0.0183
Episode:548 meanR:105.4900 R:65.0000 rate:0.1300 aloss:0.7201 eloss:-8.2803 aloss2:56.7551 exploreP:0.0183
Episode:549 meanR:104.4600 R:12.0000 rate:0.0240 aloss:0.6633 eloss:-8.2912 aloss2:58.6264 exploreP:0.0183
Episode:550 meanR:103.7300 R:11.0000 rate:0.0220 aloss:0.7714 eloss:-10.2093 aloss2:55.5725 exploreP:0.0183
Episode:551 meanR:103.3600 R:72.0000 rate:0.1440 aloss:0.7131 eloss:-8.8125 aloss2:56.6632 exploreP:0.0182
Episode:552 meanR:102.9100 R:66.0000 rate:0.1320 aloss:0.7453 eloss:-7.2079 aloss2:56.9159 exploreP:0.0182
Episode:553 meanR:102.1400 R:14.0000 rate:0.0280 aloss:0.6159 eloss:-10.6046 aloss2:57.1393 exploreP:0.0181
Episode:554 meanR:102.0400 R:87.

Episode:622 meanR:66.2000 R:12.0000 rate:0.0240 aloss:0.6493 eloss:-9.6675 aloss2:59.0973 exploreP:0.0154
Episode:623 meanR:65.8300 R:91.0000 rate:0.1820 aloss:0.6588 eloss:-10.2346 aloss2:58.4451 exploreP:0.0154
Episode:624 meanR:64.9200 R:10.0000 rate:0.0200 aloss:0.6661 eloss:-8.7211 aloss2:58.9798 exploreP:0.0154
Episode:625 meanR:65.0400 R:101.0000 rate:0.2020 aloss:0.6488 eloss:-9.8899 aloss2:58.6433 exploreP:0.0153
Episode:626 meanR:65.4400 R:209.0000 rate:0.4180 aloss:0.6734 eloss:-8.7340 aloss2:58.0701 exploreP:0.0152
Episode:627 meanR:64.4900 R:10.0000 rate:0.0200 aloss:0.6670 eloss:-8.8454 aloss2:57.8006 exploreP:0.0152
Episode:628 meanR:65.4700 R:181.0000 rate:0.3620 aloss:0.6685 eloss:-9.1713 aloss2:58.5855 exploreP:0.0151
Episode:629 meanR:65.4900 R:97.0000 rate:0.1940 aloss:0.7023 eloss:-9.8907 aloss2:58.6785 exploreP:0.0150
Episode:630 meanR:64.9100 R:10.0000 rate:0.0200 aloss:0.6697 eloss:-10.0848 aloss2:59.5000 exploreP:0.0150
Episode:631 meanR:64.2500 R:10.0000 rate:

Episode:699 meanR:63.2500 R:90.0000 rate:0.1800 aloss:0.6812 eloss:-6.6792 aloss2:59.7482 exploreP:0.0133
Episode:700 meanR:63.7200 R:124.0000 rate:0.2480 aloss:0.6747 eloss:-9.9600 aloss2:59.7926 exploreP:0.0132
Episode:701 meanR:62.5400 R:9.0000 rate:0.0180 aloss:0.7449 eloss:-8.7753 aloss2:59.8128 exploreP:0.0132
Episode:702 meanR:62.3000 R:95.0000 rate:0.1900 aloss:0.6457 eloss:-9.8200 aloss2:60.0942 exploreP:0.0132
Episode:703 meanR:62.7400 R:132.0000 rate:0.2640 aloss:0.6657 eloss:-8.0133 aloss2:59.5552 exploreP:0.0132
Episode:704 meanR:61.8100 R:10.0000 rate:0.0200 aloss:0.7019 eloss:-10.7925 aloss2:59.8379 exploreP:0.0132
Episode:705 meanR:61.7600 R:122.0000 rate:0.2440 aloss:0.6927 eloss:-9.3496 aloss2:60.2047 exploreP:0.0131
Episode:706 meanR:60.8900 R:10.0000 rate:0.0200 aloss:0.7523 eloss:-6.7877 aloss2:58.6935 exploreP:0.0131
Episode:707 meanR:60.8900 R:10.0000 rate:0.0200 aloss:0.7877 eloss:-9.5453 aloss2:60.5522 exploreP:0.0131
Episode:708 meanR:60.8800 R:10.0000 rate:0.

Episode:776 meanR:70.6800 R:84.0000 rate:0.1680 aloss:0.6461 eloss:-9.6662 aloss2:61.8955 exploreP:0.0118
Episode:777 meanR:70.4100 R:124.0000 rate:0.2480 aloss:0.6670 eloss:-8.3296 aloss2:61.9114 exploreP:0.0118
Episode:778 meanR:71.2900 R:98.0000 rate:0.1960 aloss:0.6743 eloss:-7.7412 aloss2:62.5761 exploreP:0.0118
Episode:779 meanR:72.2200 R:102.0000 rate:0.2040 aloss:0.6375 eloss:-9.1357 aloss2:62.2347 exploreP:0.0118
Episode:780 meanR:72.8700 R:174.0000 rate:0.3480 aloss:0.6266 eloss:-9.2609 aloss2:62.2232 exploreP:0.0117
Episode:781 meanR:74.0400 R:127.0000 rate:0.2540 aloss:0.6363 eloss:-10.8420 aloss2:62.7663 exploreP:0.0117
Episode:782 meanR:74.9900 R:105.0000 rate:0.2100 aloss:0.6714 eloss:-9.3163 aloss2:62.6536 exploreP:0.0117
Episode:783 meanR:74.8100 R:106.0000 rate:0.2120 aloss:0.6437 eloss:-10.4663 aloss2:62.3853 exploreP:0.0117
Episode:784 meanR:74.8200 R:10.0000 rate:0.0200 aloss:0.6116 eloss:-5.5964 aloss2:62.0355 exploreP:0.0117
Episode:785 meanR:73.8000 R:9.0000 rat

Episode:853 meanR:95.4700 R:104.0000 rate:0.2080 aloss:0.5954 eloss:-11.5338 aloss2:66.7973 exploreP:0.0108
Episode:854 meanR:95.2500 R:100.0000 rate:0.2000 aloss:0.5709 eloss:-9.4215 aloss2:67.0525 exploreP:0.0108
Episode:855 meanR:95.2900 R:96.0000 rate:0.1920 aloss:0.5887 eloss:-11.2631 aloss2:66.8116 exploreP:0.0108
Episode:856 meanR:96.7400 R:156.0000 rate:0.3120 aloss:0.5641 eloss:-10.6984 aloss2:66.7209 exploreP:0.0108
Episode:857 meanR:96.8600 R:106.0000 rate:0.2120 aloss:0.5852 eloss:-8.3875 aloss2:68.0359 exploreP:0.0108
Episode:858 meanR:97.9300 R:117.0000 rate:0.2340 aloss:0.6227 eloss:-10.8828 aloss2:66.7400 exploreP:0.0108
Episode:859 meanR:98.0200 R:116.0000 rate:0.2320 aloss:0.5822 eloss:-9.9912 aloss2:67.3349 exploreP:0.0108
Episode:860 meanR:98.0300 R:95.0000 rate:0.1900 aloss:0.5608 eloss:-6.3746 aloss2:66.5824 exploreP:0.0108
Episode:861 meanR:97.9800 R:91.0000 rate:0.1820 aloss:0.5613 eloss:-10.4849 aloss2:67.2072 exploreP:0.0108
Episode:862 meanR:99.2700 R:139.000

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
